# Convolutional Neural Network in Pytorch
- following this guide: `https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/`

In [2]:
from torch.nn import (
    Module, 
    Conv2d, 
    Linear, 
    MaxPool2d, 
    ReLU, 
    LogSoftmax
)
from torch import flatten

/home/mmazeem/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
class LeNet(Module):
    def __init__(self, num_channels, classes):
        super().__init__()

        # First set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(
            in_channels=num_channels,   # Channels in input image
            out_channels=20,            # Number of filters
            kernel_size=(5,5)           # Size of each filter
        )
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2,2), stride=(2,2))

        # Second set of CONV => RELU => POOL lyaers
        self.conv2 = Conv2d(
            in_channels=20,             # Output channels from first conv
            out_channels=50,            # Number of filters
            kernel_size=(5,5)           # Size of each filter
        )
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2,2), stride=(2,2))

        # FC => RELU layers
        self.fc1 = Linear(
            in_features=800,            # num of input = flattened conv output
            out_features=500            # num of hidden neurons
        )
        self.relu3 = ReLU()

        # Softmax classifier
        self.fc2 = Linear(
            in_features=500,            # num of hidden neurons
            out_features=classes        # num of output neurons
        )
        self.log_softmax = LogSoftmax(dim=1)


    def forward(self, x):
        print(f'orig:\t{x.shape}')
        # First set of conv layers
        x = self.conv1(x)
        print(f'conv1:\t{x.shape}')
        x = self.relu1(x)
        print(f'relu1:\t{x.shape}')
        x = self.maxpool1(x)
        print(f'maxpool1:\t{x.shape}')

        # Second set of conv layers
        x = self.conv2(x)
        print(f'conv2:\t{x.shape}')
        x = self.relu2(x)
        print(f'relu2:\t{x.shape}')
        x = self.maxpool2(x)
        print(f'maxpool2:\t{x.shape}')

        # Flatten output and pass it to dense layers
        x = flatten(x, 1)
        print(f'flatten:\t{x.shape}')
        x = self.fc1(x)
        print(f'fc1:\t{x.shape}')
        x = self.relu3(x)
        print(f'relu3:\t{x.shape}')

        # Softmax classifier
        x = self.fc2(x)
        print(f'fc2:\t{x.shape}')
        output = self.log_softmax(x)
        print(f'log softmax:\t{x.shape}')
        print()

        return output

## Train

In [20]:
import matplotlib
matplotlib.use("Agg")

from sklearn.metrics import classification_report
from torch.utils.data import (random_split, DataLoader)
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch
import time

In [21]:
# Training hyperparameters
init_lr = 1e-3
batch_size = 64
epochs = 10

# train and val split
train_split = 0.75
val_split = 1 - train_split

# Get device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Prepare Dataset

In [22]:
train_data = KMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = KMNIST(root="data", train=False, download=True, transform=ToTensor())

num_train_samples = int(len(train_data) * train_split)
num_val_samples = int(len(train_data) * val_split)
(train_data, val_data) = random_split(
    train_data,
	[num_train_samples, num_val_samples],
	generator=torch.Generator().manual_seed(42)
)

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

train_steps = len(train_dataloader.dataset) // batch_size
val_steps = len(val_dataloader.dataset) // batch_size

### Train

In [29]:
model = LeNet(
    num_channels=1,
    classes=len(train_data.dataset.classes)
).to(device)

opt = Adam(model.parameters(), lr=init_lr)
loss_fn = nn.NLLLoss()

# Training History
H = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

In [30]:
for e in range(epochs):
    model.train()

    total_train_loss = 0
    total_val_loss = 0

    train_correct = 0
    val_correct = 0

    for (x,y) in train_dataloader:
        (x,y) = (x.to(device), y.to(device))

        pred = model(x)
        loss = loss_fn(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_train_loss += loss
        train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    with torch.no_grad():
        model.eval()

        for (x,y) in val_dataloader:
            (x,y) = (x.to(device), y.to(device))
            pred = model(x)
            total_val_loss += loss_fn(pred, y)

            val_correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    # calculate the average training and validation loss
    avg_train_loss = total_train_loss / train_steps
    avg_val_loss = total_val_loss / val_steps
    # calculate the training and validation accuracy
    train_correct = train_correct / len(train_dataloader.dataset)
    val_correct = val_correct / len(val_dataloader.dataset)
    # update our training history
    H["train_loss"].append(avg_train_loss.cpu().detach().numpy())
    H["train_acc"].append(train_correct)
    H["val_loss"].append(avg_val_loss.cpu().detach().numpy())
    H["val_acc"].append(val_correct)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, epochs))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avg_train_loss, train_correct))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(avg_val_loss, val_correct))

conv1:	torch.Size([64, 20, 24, 24])
relu1:	torch.Size([64, 20, 24, 24])
maxpool1:	torch.Size([64, 20, 12, 12])
conv2:	torch.Size([64, 50, 8, 8])
relu2:	torch.Size([64, 50, 8, 8])
maxpool2:	torch.Size([64, 50, 4, 4])
flatten:	torch.Size([64, 800])
fc1:	torch.Size([64, 500])
relu3:	torch.Size([64, 500])
fc2:	torch.Size([64, 10])
log softmax:	torch.Size([64, 10])
conv1:	torch.Size([64, 20, 24, 24])
relu1:	torch.Size([64, 20, 24, 24])
maxpool1:	torch.Size([64, 20, 12, 12])
conv2:	torch.Size([64, 50, 8, 8])
relu2:	torch.Size([64, 50, 8, 8])
maxpool2:	torch.Size([64, 50, 4, 4])
flatten:	torch.Size([64, 800])
fc1:	torch.Size([64, 500])
relu3:	torch.Size([64, 500])
fc2:	torch.Size([64, 10])
log softmax:	torch.Size([64, 10])
conv1:	torch.Size([64, 20, 24, 24])
relu1:	torch.Size([64, 20, 24, 24])
maxpool1:	torch.Size([64, 20, 12, 12])
conv2:	torch.Size([64, 50, 8, 8])
relu2:	torch.Size([64, 50, 8, 8])
maxpool2:	torch.Size([64, 50, 4, 4])
flatten:	torch.Size([64, 800])
fc1:	torch.Size([64, 500])


KeyboardInterrupt: 

In [26]:
with torch.no_grad():
    model.eval()
    preds = []
    for (x,y) in test_dataloader:
        x = x.to(device)
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy())

In [27]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')
# serialize the model to disk
torch.save(model, 'model/model.pt')